<a href="https://colab.research.google.com/github/jumbokh/csu1112-class/blob/main/class/%E5%86%92%E9%9A%AA31_%E8%AE%93%E6%88%91%E5%80%91%E5%81%9A%E6%AD%8C%E8%A9%9E%E7%94%A2%E7%94%9F%E5%99%A8%E7%B6%B2%E8%B7%AFApp!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 繁體簡體互換

In [1]:
!pip install opencc-python-reimplemented

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 9.6 MB/s eta 0:00:00


In [2]:
from opencc import OpenCC

In [3]:
OpenCC('t2s').convert("現在人工智慧發展很快。")

'现在人工智慧发展很快。'

In [4]:
t2s = OpenCC('tw2sp').convert
s2t = OpenCC('s2twp').convert

In [5]:
t2s('現在人工智慧發展很快。')

'现在人工智能发展很快。'

In [6]:
s2t("我还是习惯有线鼠标")

'我還是習慣有線滑鼠'

###2. 中文版的一行文生成器

In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.1 MB/s eta 0:00:00


In [8]:
from transformers import pipeline

In [9]:
generator = pipeline("text-generation", model="mymusise/gpt2-medium-chinese")

Some weights of the model checkpoint at mymusise/gpt2-medium-chinese were not used when initializing GPT2LMHeadModel: ['score.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [10]:
text = t2s("我是一位正在學習程式的學生，我計畫明年")

In [11]:
[egg] = generator(text, max_length=100)

In [12]:
result = s2t(''.join(egg['generated_text'].split()))

In [13]:
result

'我是一位正在學習程式的學生，我計劃明年早點前送他們到南方去；雖然這段路線不很廣闊，但它還和以往在那裡相當的遠距離。而且每次我都不再回北京就被人帶著車從這兒發出來了；另外也有可能去其它地方或山mar水灣。在'

### 3. 原版的 BERT 做中文填字


In [14]:
fill_mask = pipeline('fill-mask', model='bert-base-chinese')

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
fill_mask('生命的真諦是[MASK]。')

[{'score': 0.2658056914806366,
  'token': 2695,
  'token_str': '愛',
  'sequence': '生 命 的 真 諦 是 愛 。'},
 {'score': 0.039826635271310806,
  'token': 872,
  'token_str': '你',
  'sequence': '生 命 的 真 諦 是 你 。'},
 {'score': 0.030924538150429726,
  'token': 5401,
  'token_str': '美',
  'sequence': '生 命 的 真 諦 是 美 。'},
 {'score': 0.030239388346672058,
  'token': 3634,
  'token_str': '此',
  'sequence': '生 命 的 真 諦 是 此 。'},
 {'score': 0.02728997729718685,
  'token': 8038,
  'token_str': '：',
  'sequence': '生 命 的 真 諦 是 ： 。'}]

In [16]:
def fill(text):
    result = fill_mask(text)
    generated_text = []
    for d in result:
        s = ''.join(d['sequence'].split())
        print(s)
        generated_text.append(s)

In [17]:
fill('我天天都會喝一杯[MASK]。')

我天天都會喝一杯酒。
我天天都會喝一杯茶。
我天天都會喝一杯水。
我天天都會喝一杯的。
我天天都會喝一杯奶。


### 4. 閱讀測驗型的問答

In [18]:
question_answerer = pipeline("question-answering", model='uer/roberta-base-chinese-extractive-qa')

In [19]:
context = '''報導隨後介紹本屆奧運金牌得主郭婞淳先前的善舉，除提到郭婞淳名字來自「倖存」一意，也提到郭婞淳在2016年購買救護車送給羅東聖母醫院，後再轉贈離島澎湖惠民醫院的故事。

報導取用澎湖縣長賴峰偉臉書發文的照片，當時郭婞淳奪金後，惠民醫院的醫護人員也覺得與有榮焉。文末則提到郭婞淳是台灣原住民，認為她以阿美族身分一舉奪金，同樣備受關注。'''

我們寫個函式, 把繁體轉簡體, 送進 model 看看答案是什麼。最後回轉簡體轉繁體的答案。

In [20]:
def QA(question, context):
    c = t2s(context)
    q = t2s(question)
    result = question_answerer(question=q, context=c)
    ans = s2t(result['answer'])
    return ans

In [21]:
question = "郭婞淳捐贈的救護車，最後是在哪個醫院？"
QA(question, context)

'離島澎湖惠民醫院'

In [22]:
question = "郭婞淳是哪一個原住民族的人？"
QA(question, context)

'阿美族'

### 5. 歌詞產生器

In [23]:
generator = pipeline("text-generation", model="uer/gpt2-chinese-lyric")

In [24]:
t2s('來到一個陌生又熟悉的城市，')

'来到一个陌生又熟悉的城市，'

In [25]:
lyric_s = generator('来到一个陌生又熟悉的城市，', max_length=100)[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [26]:
''.join(lyric_s.split())

'来到一个陌生又熟悉的城市，不由自主的闯进了她的视线，曾经一起漫步的兄弟，慢慢的远离了这里，现在只留下了我一个人，只能够再一次选择把她捧在手心，不由自主的想起了那段情义，她们在我心里不曾忘记，就像有首歌'

In [27]:
s2t(''.join(lyric_s.split()))

'來到一個陌生又熟悉的城市，不由自主的闖進了她的視線，曾經一起漫步的兄弟，慢慢的遠離了這裡，現在只留下了我一個人，只能夠再一次選擇把她捧在手心，不由自主的想起了那段情義，她們在我心裡不曾忘記，就像有首歌'

### 6. 做一個歌詞產生器的網路App！

In [28]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.6/136.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 k

In [29]:
import gradio as gr

In [30]:
def lyric_generator(text):
    egg = t2s(text)
    lyric = generator(egg, max_length=100)[0]['generated_text']
    return s2t(''.join(lyric.split()))

In [31]:
iface = gr.Interface(lyric_generator,
                     inputs='text',
                     outputs='text',
                     title='歌詞產生器',
                     description='輸入一小段你的作詞靈感，就可以自動產生歌詞！')

In [32]:
iface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>